In [1]:
pip install google-generativeai faiss-cpu sentence-transformers PyPDF2 python-docx 


   ---------------------------------------- 0.0/14.9 MB ? eta -:--:--
   ------------ --------------------------- 4.7/14.9 MB 28.6 MB/s eta 0:00:01
   ----------------------- ---------------- 8.9/14.9 MB 26.4 MB/s eta 0:00:01
   ---------------------------------------  14.7/14.9 MB 25.6 MB/s eta 0:00:01
   ---------------------------------------- 14.9/14.9 MB 24.0 MB/s eta 0:00:00


In [5]:
pip install tf-keras


   ---------------------------------------- 0.0/1.7 MB ? eta -:--:--
   ---------------------------------------- 1.7/1.7 MB 18.8 MB/s eta 0:00:00


In [1]:
import os
import faiss
import numpy as np
from typing import List, Dict, Any
import google.generativeai as genai
from sentence_transformers import SentenceTransformer
import PyPDF2
from docx import Document
import re
import pickle

class DocumentProcessor:
    """Handles PDF and DOCX document processing"""
    
    @staticmethod
    def extract_text_from_pdf(file_path: str) -> str:
        """Extract text from PDF file"""
        text = ""
        try:
            with open(file_path, 'rb') as file:
                pdf_reader = PyPDF2.PdfReader(file)
                for page in pdf_reader.pages:
                    text += page.extract_text() + "\n"
        except Exception as e:
            print(f"Error reading PDF: {e}")
        return text
    
    @staticmethod
    def extract_text_from_docx(file_path: str) -> str:
        """Extract text from DOCX file"""
        text = ""
        try:
            doc = Document(file_path)
            for paragraph in doc.paragraphs:
                text += paragraph.text + "\n"
        except Exception as e:
            print(f"Error reading DOCX: {e}")
        return text
    
    @staticmethod
    def chunk_text(text: str, chunk_size: int = 1000, overlap: int = 200) -> List[str]:
        """Split text into overlapping chunks"""
        # Clean the text
        text = re.sub(r'\s+', ' ', text.strip())
        
        chunks = []
        start = 0
        
        while start < len(text):
            end = start + chunk_size
            
            if end < len(text):
                sentence_ends = ['.', '!', '?']
                for i in range(end, max(start + chunk_size // 2, end - 100), -1):
                    if text[i] in sentence_ends and i + 1 < len(text) and text[i + 1] == ' ':
                        end = i + 1
                        break
            
            chunk = text[start:end].strip()
            if chunk:
                chunks.append(chunk)
            
            start = end - overlap
            
        return chunks

class RAGSystem:
    """Main RAG system class"""
    
    def __init__(self, gemini_api_key: str, embedding_model: str = "all-MiniLM-L6-v2"):
        """
        Initialize RAG system
        
        Args:
            gemini_api_key: Google Gemini API key
            embedding_model: Sentence transformer model for embeddings
        """
        genai.configure(api_key=gemini_api_key)
        self.gemini_model = genai.GenerativeModel('gemini-2.0-flash')
        
        self.embedding_model = SentenceTransformer(embedding_model)
        self.embedding_dim = self.embedding_model.get_sentence_embedding_dimension()
        
        self.index = faiss.IndexFlatIP(self.embedding_dim)  # Inner product for cosine similarity
        
        self.documents = []
        self.metadata = []
        
        self.doc_processor = DocumentProcessor()
        
    def add_document(self, file_path: str) -> None:
        """
        Add a document to the knowledge base
        
        Args:
            file_path: Path to PDF or DOCX file
        """
        file_ext = file_path.lower().split('.')[-1]
        
        # Extract text based on file type
        if file_ext == 'pdf':
            text = self.doc_processor.extract_text_from_pdf(file_path)
        elif file_ext == 'docx':
            text = self.doc_processor.extract_text_from_docx(file_path)
        else:
            raise ValueError(f"Unsupported file type: {file_ext}")
        
        if not text.strip():
            raise ValueError(f"No text extracted from {file_path}")
        
        # Chunk the text
        chunks = self.doc_processor.chunk_text(text)
        
        # Generate embeddings
        embeddings = self.embedding_model.encode(chunks)
        
        # Normalize embeddings for cosine similarity
        embeddings = embeddings / np.linalg.norm(embeddings, axis=1, keepdims=True)
        
        # Add to FAISS index
        self.index.add(embeddings.astype('float32'))
        
        # Store documents and metadata
        for i, chunk in enumerate(chunks):
            self.documents.append(chunk)
            self.metadata.append({
                'source': file_path,
                'chunk_id': len(self.documents),
                'text': chunk
            })
        
        print(f"Added {len(chunks)} chunks from {file_path}")
    
    def search_similar_documents(self, query: str, k: int = 5) -> List[Dict[str, Any]]:
        """
        Search for similar documents using vector similarity
        
        Args:
            query: Search query
            k: Number of similar documents to return
            
        Returns:
            List of similar document chunks with metadata
        """
        if self.index.ntotal == 0:
            return []
        
        query_embedding = self.embedding_model.encode([query])
        query_embedding = query_embedding / np.linalg.norm(query_embedding, axis=1, keepdims=True)
        
        scores, indices = self.index.search(query_embedding.astype('float32'), k)
        
        results = []
        for score, idx in zip(scores[0], indices[0]):
            if idx < len(self.documents):
                results.append({
                    'text': self.documents[idx],
                    'score': float(score),
                    'metadata': self.metadata[idx]
                })
        
        return results
    
    def generate_answer(self, query: str, context_chunks: List[str]) -> str:
        """
        Generate answer using Gemini with retrieved context
        
        Args:
            query: User question
            context_chunks: Retrieved relevant text chunks
            
        Returns:
            Generated answer
        """
        # Combine context chunks
        context = "\n\n".join(context_chunks)
        
        # Create prompt
        prompt = f"""
Based on the following context, please answer the question. If the answer cannot be found in the context, please say so.

Context:
{context}

Question: {query}

Answer:
"""
        
        try:
            response = self.gemini_model.generate_content(prompt)
            return response.text
        except Exception as e:
            return f"Error generating response: {e}"
    
    def query(self, question: str, k: int = 5) -> Dict[str, Any]:
        """
        Main query method that retrieves relevant documents and generates answer
        
        Args:
            question: User question
            k: Number of documents to retrieve
            
        Returns:
            Dictionary containing answer, sources, and retrieved documents
        """
        # Retrieve similar documents
        similar_docs = self.search_similar_documents(question, k)
        
        if not similar_docs:
            return {
                'answer': "No relevant documents found in the knowledge base.",
                'sources': [],
                'retrieved_docs': []
            }
        
        # Extract text chunks for context
        context_chunks = [doc['text'] for doc in similar_docs]
        
        # Generate answer
        answer = self.generate_answer(question, context_chunks)
        
        # Extract unique sources
        sources = list(set([doc['metadata']['source'] for doc in similar_docs]))
        
        return {
            'answer': answer,
            'sources': sources,
            'retrieved_docs': similar_docs
        }
    
    def save_index(self, save_path: str) -> None:
        """Save the FAISS index and metadata"""
        # Save FAISS index
        faiss.write_index(self.index, f"{save_path}.faiss")
        
        # Save documents and metadata
        with open(f"{save_path}_data.pkl", 'wb') as f:
            pickle.dump({
                'documents': self.documents,
                'metadata': self.metadata
            }, f)
        
        print(f"Index saved to {save_path}")
    
    def load_index(self, load_path: str) -> None:
        """Load a previously saved FAISS index and metadata"""
        # Load FAISS index
        self.index = faiss.read_index(f"{load_path}.faiss")
        
        with open(f"{load_path}_data.pkl", 'rb') as f:
            data = pickle.load(f)
            self.documents = data['documents']
            self.metadata = data['metadata']
        
        print(f"Index loaded from {load_path}")

def main():
    GEMINI_API_KEY = "API-KEY"  
    rag = RAGSystem(GEMINI_API_KEY)
    
    try:
        rag.add_document(r"C:\Users\Bhumi\Downloads\How-to-stop-overthinking-Kindle-Book.pdf")
        # rag.add_document("document2.docx")/
        
        rag.save_index("my_rag_index")
        
        print("\n=== RAG System Ready ===")
        print("Ask questions about your documents (type 'quit' to exit):")
        
        while True:
            question = input("\nQuestion: ").strip()
            
            if question.lower() in ['quit', 'exit', 'q']:
                break
            
            if not question:
                continue
            
            # Get answer
            result = rag.query(question)
            
            print(f"\nAnswer: {result['answer']}")
            print(f"\nSources: {', '.join(result['sources'])}")
            
            # Optionally show retrieved documents
            show_docs = input("\nShow retrieved documents? (y/n): ").lower() == 'y'
            if show_docs:
                print("\nRetrieved Documents:")
                for i, doc in enumerate(result['retrieved_docs'], 1):
                    print(f"\n{i}. (Score: {doc['score']:.3f})")
                    print(f"Text: {doc['text'][:200]}...")
    
    except Exception as e:
        print(f"Error: {e}")

if __name__ == "__main__":
    main()

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

C:\Users\Bhumi\anaconda3\Lib\site-packages\huggingface_hub\file_download.py:143: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\Bhumi\.cache\huggingface\hub\models--sentence-transformers--all-MiniLM-L6-v2. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...


Show retrieved documents? (y/n):  n

Question:  who is Dr. Saloni Singh



Answer: Dr. Saloni Singh is the author of the book which provides proven ways to relieve anxiety, stress, confusion and tap into the power of a calm and clear mind.


Sources: C:\Users\Bhumi\Downloads\How-to-stop-overthinking-Kindle-Book.pdf



Show retrieved documents? (y/n):  n

Question:  who is Neil Postman



Answer: Neil Postman is mentioned in the context as the author of the quote: “Information has become a form of garbage. We don’t know what to do with it, have no control over it; don’t know how to get rid of it.”


Sources: C:\Users\Bhumi\Downloads\How-to-stop-overthinking-Kindle-Book.pdf



Show retrieved documents? (y/n):  n

Question:  exit
